In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1036.57it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1438.05it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=3, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(train_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
        print(supportLabels)

[1, 3, 2, 0, 0, 2, 3, 1, 2, 1, 3, 0]
[3, 2, 3, 0, 2, 1, 3, 2, 1, 1, 0, 0]
[1, 2, 3, 2, 0, 0, 3, 3, 1, 1, 2, 0]
[0, 2, 1, 2, 3, 1, 0, 0, 2, 3, 1, 3]
[0, 1, 3, 2, 2, 0, 3, 3, 2, 1, 0, 1]
[2, 3, 2, 0, 1, 0, 3, 1, 1, 2, 0, 3]
[3, 0, 2, 1, 1, 1, 3, 2, 0, 0, 3, 2]
[0, 3, 0, 3, 1, 3, 2, 2, 2, 1, 1, 0]


In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

# val_protomaml_loader = data.DataLoader(
#     None, batch_sampler=val_protomaml_sampler, num_workers=1
# )


In [6]:
for i in range(1):
    batch = next(iter(val_protomaml_loader))
#     print(batch)
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2]
        print(supportLabels)

[1, 1, 0, 0]
[2, 4, 4, 3, 2, 3]
[5, 6, 5, 6, 7, 7]
[8, 8, 9, 10, 9, 10]
[13, 12, 12, 11, 11, 13]
[15, 14, 15, 14]
[16, 16, 17, 17]
[18, 19, 18, 19]
